In [1]:
# %% General imports
import numpy as np
import torch

# DeepMoD stuff
from multitaskpinn import DeepMoD
from multitaskpinn.model.func_approx import NN
from multitaskpinn.model.library import Library1D
from multitaskpinn.model.constraint import LeastSquares
from multitaskpinn.model.sparse_estimators import Threshold
from multitaskpinn.training.sparsity_scheduler import TrainTestPeriodic
from multitaskpinn.utils.logger import Logger
from multitaskpinn.training.convergence import Convergence
from multitaskpinn.training import train

from deepymod.data import Dataset
from deepymod.data.burgers import BurgersDelta
from deepymod.analysis import load_tensorboard
from deepymod.data.kdv import DoubleSoliton


%load_ext autoreload
%autoreload 2

In [2]:
# Settings
#if torch.cuda.is_available():
#    device = 'cuda'
#else:
#    device = 'cpu'

device = 'cpu'

# Settings for reproducibility
np.random.seed(42)
torch.manual_seed(0)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

# Making dataset

In [3]:
# Making dataset
v = 0.1
A = 1.0

x = np.linspace(-3, 4, 10)
t = np.linspace(0.1, 5.0, 100)
x_grid, t_grid = np.meshgrid(x, t, indexing='ij')
dataset = Dataset(BurgersDelta, v=v, A=A)
X, y = dataset.create_dataset(x_grid.reshape(-1, 1), t_grid.reshape(-1, 1), n_samples=0, noise=0.2, random=True, normalize=True)
X, y = X.to(device), y.to(device)

# SBL training function

In [4]:
def train_bayes_separate(model: DeepMoD,
          data: torch.Tensor,
          target: torch.Tensor,
          optimizer,
          sparsity_scheduler,
          split = 0.8,
          exp_ID: str = None,
          log_dir: str = None,
          max_iterations: int = 10000,
          write_iterations: int = 25,
          **convergence_kwargs) -> None:
    """Trains the DeepMoD model. This function automatically splits the data set in a train and test set. 

    Args:
        model (DeepMoD):  A DeepMoD object.
        data (torch.Tensor):  Tensor of shape (n_samples x (n_spatial + 1)) containing the coordinates, first column should be the time coordinate.
        target (torch.Tensor): Tensor of shape (n_samples x n_features) containing the target data.
        optimizer ([type]):  Pytorch optimizer.
        sparsity_scheduler ([type]):  Decides when to update the sparsity mask.
        split (float, optional):  Fraction of the train set, by default 0.8.
        exp_ID (str, optional): Unique ID to identify tensorboard file. Not used if log_dir is given, see pytorch documentation.
        log_dir (str, optional): Directory where tensorboard file is written, by default None.
        max_iterations (int, optional): [description]. Max number of epochs , by default 10000.
        write_iterations (int, optional): [description]. Sets how often data is written to tensorboard and checks train loss , by default 25.
    """
    logger = Logger(exp_ID, log_dir)
    sparsity_scheduler.path = logger.log_dir # write checkpoint to same folder as tb output.
    
    # Splitting data, assumes data is already randomized
    n_train = int(split * data.shape[0])
    n_test = data.shape[0] - n_train
    data_train, data_test = torch.split(data, [n_train, n_test], dim=0)
    target_train, target_test = torch.split(target, [n_train, n_test], dim=0)

    n_samples = data_train.shape[0]
    model.t.data = -torch.var(target)
    model.b.data = -torch.var(target)
    
    s0 = torch.tensor(1e-3)
    a = torch.tensor(1.0) 
    
    # Training
    convergence = Convergence(**convergence_kwargs)
    for iteration in torch.arange(0, max_iterations):
        # ================== Training Model ============================
        prediction, time_derivs, thetas = model(data_train)

            
        MSE = torch.mean((prediction - target_train)**2, dim=0)  # loss per output
        Reg = torch.stack([torch.mean((dt - theta @ coeff_vector)**2)
                           for dt, theta, coeff_vector in zip(time_derivs, thetas, model.constraint_coeffs(scaled=False, sparse=True))])
        
        n_samples = MSE.shape[0]
        p_MSE = n_samples / 2 * torch.log(torch.sum(MSE))
        #p_reg = ((n_samples * (a + 1) + 1)/ 2 * torch.log(1 + torch.sum(Reg) / (a * s0 + torch.sum(Reg))) 
              #  - torch.lgamma((n_samples * (a + 1) + 1)/2)
              #  + torch.lgamma((n_samples * (a + 1))/2)
              #  + 0.5 * torch.log(n_samples * (a + 1))
              #  + n_samples / 2 * torch.log((a * s0 + torch.sum(Reg))/(a + 1))
              #  )
        
        p_reg = ((n_samples * (a + 1) + 1)/ 2 * torch.log(1 + torch.sum(Reg) / (a * s0 + torch.sum(Reg))) 
                + n_samples / 2 * torch.log((a * s0 + torch.sum(Reg))/(a + 1)))
   
    
        loss = p_reg + p_MSE
        # Optimizer step
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if iteration % write_iterations == 0:
            # ================== Validation costs ================
            with torch.no_grad():
                prediction_test = model.func_approx(data_test)[0]
                MSE_test = torch.mean((prediction_test - target_test)**2, dim=0)  # loss per output
         
            # ====================== Logging =======================
            _ = model.sparse_estimator(thetas, time_derivs) # calculating estimator coeffs but not setting mask
            logger(iteration, 
                   loss, MSE, Reg,
                   model.constraint_coeffs(sparse=True, scaled=True), 
                   model.constraint_coeffs(sparse=True, scaled=False),
                   model.estimator_coeffs(),
                   MSE_test=MSE_test,
                   p_MSE=p_MSE,
                   p_reg=p_reg)

            # ================== Sparsity update =============
            # Updating sparsity 
            update_sparsity = sparsity_scheduler(iteration, torch.sum(MSE_test), model, optimizer)
            if update_sparsity: 
                model.constraint.sparsity_masks = model.sparse_estimator(thetas, time_derivs)

            # ================= Checking convergence
            l1_norm = torch.sum(torch.abs(torch.cat(model.constraint_coeffs(sparse=True, scaled=True), dim=1)))
            converged = convergence(iteration, l1_norm)
            if converged:
                break
    logger.close(model)

In [ ]:
def train_bayes_separate(model: DeepMoD,
          data: torch.Tensor,
          target: torch.Tensor,
          optimizer,
          sparsity_scheduler,
          split = 0.8,
          exp_ID: str = None,
          log_dir: str = None,
          max_iterations: int = 10000,
          write_iterations: int = 25,
          **convergence_kwargs) -> None:
    """Trains the DeepMoD model. This function automatically splits the data set in a train and test set. 

    Args:
        model (DeepMoD):  A DeepMoD object.
        data (torch.Tensor):  Tensor of shape (n_samples x (n_spatial + 1)) containing the coordinates, first column should be the time coordinate.
        target (torch.Tensor): Tensor of shape (n_samples x n_features) containing the target data.
        optimizer ([type]):  Pytorch optimizer.
        sparsity_scheduler ([type]):  Decides when to update the sparsity mask.
        split (float, optional):  Fraction of the train set, by default 0.8.
        exp_ID (str, optional): Unique ID to identify tensorboard file. Not used if log_dir is given, see pytorch documentation.
        log_dir (str, optional): Directory where tensorboard file is written, by default None.
        max_iterations (int, optional): [description]. Max number of epochs , by default 10000.
        write_iterations (int, optional): [description]. Sets how often data is written to tensorboard and checks train loss , by default 25.
    """
    logger = Logger(exp_ID, log_dir)
    sparsity_scheduler.path = logger.log_dir # write checkpoint to same folder as tb output.
    
    # Splitting data, assumes data is already randomized
    n_train = int(split * data.shape[0])
    n_test = data.shape[0] - n_train
    data_train, data_test = torch.split(data, [n_train, n_test], dim=0)
    target_train, target_test = torch.split(target, [n_train, n_test], dim=0)

    n_samples = data_train.shape[0]
    model.t.data = -torch.var(target)
    model.b.data = -torch.var(target)
    
    s0 = torch.tensor(1e-3)
    a = torch.tensor(1.0) 
    
    # Training
    convergence = Convergence(**convergence_kwargs)
    for iteration in torch.arange(0, max_iterations):
        # ================== Training Model ============================
        prediction, time_derivs, thetas = model(data_train)

            
        MSE = torch.mean((prediction - target_train)**2, dim=0)  # loss per output
        Reg = torch.stack([torch.mean((dt - theta @ coeff_vector)**2)
                           for dt, theta, coeff_vector in zip(time_derivs, thetas, model.constraint_coeffs(scaled=False, sparse=True))])
        
        n_samples = MSE.shape[0]
        p_MSE = n_samples / 2 * torch.log(torch.sum(MSE))
        #p_reg = ((n_samples * (a + 1) + 1)/ 2 * torch.log(1 + torch.sum(Reg) / (a * s0 + torch.sum(Reg))) 
              #  - torch.lgamma((n_samples * (a + 1) + 1)/2)
              #  + torch.lgamma((n_samples * (a + 1))/2)
              #  + 0.5 * torch.log(n_samples * (a + 1))
              #  + n_samples / 2 * torch.log((a * s0 + torch.sum(Reg))/(a + 1))
              #  )
        
        p_reg = ((n_samples * (a + 1) + 1)/ 2 * torch.log(1 + torch.sum(Reg) / (a * s0 + torch.sum(Reg))) 
                + n_samples / 2 * torch.log((a * s0 + torch.sum(Reg))/(a + 1)))
   
    
        loss = p_reg + p_MSE
        # Optimizer step
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if iteration % write_iterations == 0:
            # ================== Validation costs ================
            with torch.no_grad():
                prediction_test = model.func_approx(data_test)[0]
                MSE_test = torch.mean((prediction_test - target_test)**2, dim=0)  # loss per output
         
            # ====================== Logging =======================
            _ = model.sparse_estimator(thetas, time_derivs) # calculating estimator coeffs but not setting mask
            logger(iteration, 
                   loss, MSE, Reg,
                   model.constraint_coeffs(sparse=True, scaled=True), 
                   model.constraint_coeffs(sparse=True, scaled=False),
                   model.estimator_coeffs(),
                   MSE_test=MSE_test,
                   p_MSE=p_MSE,
                   p_reg=p_reg)

            # ================== Sparsity update =============
            # Updating sparsity 
            update_sparsity = sparsity_scheduler(iteration, torch.sum(MSE_test), model, optimizer)
            if update_sparsity: 
                model.constraint.sparsity_masks = model.sparse_estimator(thetas, time_derivs)

            # ================= Checking convergence
            l1_norm = torch.sum(torch.abs(torch.cat(model.constraint_coeffs(sparse=True, scaled=True), dim=1)))
            converged = convergence(iteration, l1_norm)
            if converged:
                break
    logger.close(model)

In [5]:
network = NN(2, [30, 30, 30, 30, 30], 1)
library = Library1D(poly_order=2, diff_order=3) # Library function
estimator = Threshold(0.1) # Sparse estimator 
constraint = LeastSquares() # How to constrain
model = DeepMoD(network, library, estimator, constraint, 12).to(device) # Putting it all in the model

sparsity_scheduler = TrainTestPeriodic(periodicity=50, patience=500, delta=1e-6) # in terms of write iterations
optimizer = torch.optim.Adam(model.parameters(), betas=(0.99, 0.99), amsgrad=True, lr=1e-3) # Defining optimizer

In [6]:
train_bayes(model, X, y, optimizer, sparsity_scheduler, exp_ID='bayes_1e3_prior_long', split=0.8, write_iterations=25, max_iterations=50000, delta=1e-3, patience=1e8) 

 22725  MSE: 1.51e-03  Reg: 5.38e-07  L1: 1.22e+00 

KeyboardInterrupt: 

In [10]:
train(model, X, y, optimizer, sparsity_scheduler, exp_ID='baseline', split=0.8, write_iterations=25, max_iterations=50000, delta=1e-3, patience=1e8) 

 49975  MSE: 1.49e-03  Reg: 5.70e-06  L1: 1.37e+00 Algorithm converged. Writing model to disk.


# KDV

In [15]:
# Making data
x = np.linspace(-10, 10, 256)
t = np.linspace(0.0, 2.0, 51)

x_grid, t_grid = np.meshgrid(x, t, indexing='ij')
dataset = Dataset(DoubleSoliton, c=[5.0, 1.0], x0=[-5.0, -1.0])

X, y = dataset.create_dataset(x_grid.reshape(-1, 1), t_grid.reshape(-1, 1), n_samples=0, noise=0.0, normalize=False, random=False)

X_train, y_train = dataset.create_dataset(x_grid.reshape(-1, 1), t_grid.reshape(-1, 1), n_samples=1000, noise=0.2, normalize=False, random=True)
X_train, y_train = X_train.to(device), y_train.to(device)

In [28]:
network = NN(2, [30, 30, 30, 30, 30], 1)
library = Library1D(poly_order=2, diff_order=3) # Library function
estimator = Threshold(0.1) # Sparse estimator 
constraint = LeastSquares() # How to constrain
model = DeepMoD(network, library, estimator, constraint, 12).to(device) # Putting it all in the model

sparsity_scheduler = TrainTestPeriodic(periodicity=50, patience=200, delta=1e-6) # in terms of write iterations
optimizer = torch.optim.Adam(model.parameters(), betas=(0.99, 0.99), amsgrad=True, lr=1e-3) # Defining optimizer

In [31]:
train_bayes(model, X_train, y_train, optimizer, sparsity_scheduler, exp_ID='KDV_bayes_2nd', split=0.8, write_iterations=25, max_iterations=50000, delta=1e-3, patience=1e8) 

 18025  MSE: 2.04e-02  Reg: 1.11e-06  L1: 1.77e+00 

KeyboardInterrupt: 

In [19]:
train(model, X_train, y_train, optimizer, sparsity_scheduler, exp_ID='KDV_normal', split=0.8, write_iterations=25, max_iterations=50000, delta=1e-3, patience=1e8) 

 17100  MSE: 1.14e-02  Reg: 5.24e-05  L1: 2.63e+00 

KeyboardInterrupt: 